# v0.6.0 Ergonomic Features Demo

This notebook demonstrates the new ergonomic enhancements introduced in v0.6.0.

1. **Pythonic Query Expressions**: `Person.age > 30`
2. **Fluent Graph Builder**: `.out().in_()`
3. **Magic Relation Accessors**: `alice.rel.knows`
4. **Decorator-based Signals**: `@pre_save`


In [3]:
from surrealengine import Document, StringField, IntField, create_connection
from surrealengine.signals import pre_save, receiver

# Setup connection (optional for query building demo, required for execution)
CONNECTION_URL = "ws://db:8000/rpc"
conn = create_connection(
    url=CONNECTION_URL,
    namespace="test_ns",
    database="test_db",
    username="root",
    password="root",
    make_default=True
)
await conn.connect()

## 1. Pythonic Query Expressions
You can now use Python operators `>` `<` `==` `!=` `&` `|` `~` to build queries.

In [4]:
class Person(Document):
    name = StringField()
    age = IntField()
    
    class Meta:
        collection = "person"

# Build a query
query = (Person.age > 30) & (Person.name.startswith("A"))
print(f"Where clause: {query.to_where_clause()}")

# Use in filter
qs = Person.objects.filter(query)
print(qs._build_query())

Where clause: (age > 30) AND (string::starts_with(name, "A"))
SELECT * FROM person WHERE (age > 30) AND (string::starts_with(name, "A"))


## 2. Fluent Graph Builder
Traverse logic using `.out()`, `.in_()`, and `.both()`.

In [5]:
qs = Person.objects.out("knows").out("person")
# Generated query will include traversal SELECT id, ->knows->person AS traversed FROM person
await qs.all()

[]

## 3. Magic Relation Accessors
Access relations directly from a document instance via `.rel`.

In [ ]:
p = Person(id="person:alice", name="Alice")

# Start traversal from Alice
friends_qs = p.rel.knows
# Equivalent to: Person.objects.filter(id="person:alice").out("knows")

# Chain further
friends_of_friends = p.rel.knows.out("person").out("knows")

## 4. Decorator-based Signals
Define signal handlers directly inside your Document class.

In [ ]:
class User(Document):
    username = StringField()
    
    @receiver(pre_save)
    def validate_username(self, **kwargs):
        print(f"Validating username for {self.username}...")
        if self.username == "admin":
            raise ValueError("Cannot use admin username")

u = User(username="user1")
# await u.save() # This would trigger the signal